### User-based collaborative filtering recommendations

1. Import bibliotek

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

C:\Users\dabro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2. Wczytanie danych

In [3]:
path = 'C:\\Users\\aldabrow\\Desktop\\PowerBI Excercises\\INZ\\data\\archive\\'

In [4]:
df_movie = pd.read_csv(path + "movies_metadata.csv",  engine='python' , encoding='utf8', error_bad_lines=False)
df_movie.head()

C:\Users\dabro\AppData\Local\Temp\ipykernel_20880\3838205454.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_movie = pd.read_csv(path + "movies_metadata.csv",  engine='python' , encoding='utf8', error_bad_lines=False)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\aldabrow\\Desktop\\PowerBI Excercises\\INZ\\data\\archive\\movies_metadata.csv'

In [ ]:
df_rating = pd.read_csv(path + "ratings.csv")
df_rating.head()

MemoryError: Unable to allocate 1.00 MiB for an array with shape (131072,) and data type int64

3. Transformacja i oczyszczenie danych

In [ ]:
df_movie = df_movie[['id', 'title']]
df_movie = df_movie.rename(columns={'id': 'movieId'})
df_movie.shape

(45466, 2)

In [ ]:
df_rating = df_rating.drop(columns=['timestamp'])
df_rating.shape

(26024289, 3)

In [ ]:
#convert the movieId column in the df_movie dataframe to numeric
df_movie['movieId'] = pd.to_numeric(df_movie['movieId'], errors='coerce')

#merge the two dataframes on the movieId column and id column
df_movie_rating = pd.merge(df_movie, df_rating, on='movieId')
df_movie_rating.head()

,movieId,title,userId,rating
0,862.0,Toy Story,1923,3.0
1,862.0,Toy Story,2103,5.0
2,862.0,Toy Story,5380,1.0
3,862.0,Toy Story,6177,4.0
4,862.0,Toy Story,6525,4.0


In [ ]:
df_movie_rating.shape

(11437637, 4)

In [ ]:
#drop the rows with null values
df_movie_rating = df_movie_rating.dropna()
df_movie_rating.shape

(11437637, 4)

In [ ]:
movie_rank = df_movie_rating.groupby(['title']) \
  .agg({'userId' : 'count', 'rating' : 'sum'}) \
  .rename(columns={"userId" : 'totalUniqueUsers', "rating" : "totalMovieRating"}) \
  .sort_values(['totalMovieRating'], ascending=False)

movie_rank['avgUserRat'] = movie_rank['totalMovieRating'] / movie_rank['totalUniqueUsers']
movie_rank.head()

,totalUniqueUsers,totalMovieRating,avgUserRat
title,,,
The Million Dollar Hotel,91082,403403.5,4.429015
Terminator 3: Rise of the Machines,87901,366545.0,4.169975
Solaris,84318,349836.5,4.149013
The 39 Steps,77045,318373.0,4.132299
Once Were Warriors,67662,288682.0,4.266531


In [ ]:
df_movie_rating = df_movie_rating.join(movie_rank, on='title', how='inner') \
    .sort_values(['rating'], ascending=False)
df_movie_rating.head()

MemoryError: Unable to allocate 87.3 MiB for an array with shape (11437637, 1) and data type float64

Zmniejszenie zbioru danych o użytkowników którzy ocenili mniej niż 50 filmów i filmów ktore zostały ocenione mniej niż 100 razy

In [ ]:
#smaller dataset by users who have rated a minimum number of movies 
filtered_users = df_movie_rating['userId'].value_counts()
filtered_users = filtered_users[filtered_users >= 50].index
df_movie_rating = df_movie_rating[df_movie_rating['userId'].isin(filtered_users)]

filtered_movies = df_movie_rating['movieId'].value_counts()
filtered_movies = filtered_movies[filtered_movies >= 100].index
df_movie_rating = df_movie_rating[df_movie_rating['movieId'].isin(filtered_movies)]
df_movie_rating.shape

(8268572, 7)

Normalizacja ratingu do przedzialu [0,1]

In [ ]:
#scale the rating column to be between 0 and 1
min_rating = df_movie_rating['rating'].min()
max_rating = df_movie_rating['rating'].max()

# Normalize ratings to the range [0, 1]
df_movie_rating['normalized_rating'] = (df_movie_rating['rating'] - min_rating) / (max_rating - min_rating)
df_movie_rating.head()

,movieId,title,userId,rating,totalUniqueUsers,totalMovieRating,avgUserRat,normalized_rating
5812604,153.0,Lost in Translation,238314,5.0,38187,110989.5,2.906473,1.0
1736891,2058.0,Addicted to Love,177528,5.0,10200,36550.5,3.583382,1.0
6067332,2297.0,Kafka,11336,5.0,4842,16047.5,3.314230,1.0
6067333,2297.0,Kafka,11379,5.0,4842,16047.5,3.314230,1.0
6067334,2297.0,Kafka,11443,5.0,4842,16047.5,3.314230,1.0


5. Tabela przestawna user-item

In [ ]:
#pivot the dataframe to get the movie titles as the columns and the userId as the rows and the ratings as the values
df_movie_rating_pivot = df_movie_rating.pivot_table(index='userId', columns='title', values='normalized_rating')
df_movie_rating_pivot.head()

title,!Women Art Revolution,$5 a Day,'Gator Bait,'Twas the Night Before Christmas,...And the Pursuit of Happiness,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡A volar joven!,À nos amours,Ödipussi,Şaban Oğlu Şaban,Šíleně smutná princezna
userId,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_movie_rating_pivot.shape

(61472, 2924)

In [ ]:
#fill the null values with 0
df_movie_rating_pivot.fillna(0, inplace=True)
df_movie_rating_pivot.head()

title,!Women Art Revolution,$5 a Day,'Gator Bait,'Twas the Night Before Christmas,...And the Pursuit of Happiness,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡A volar joven!,À nos amours,Ödipussi,Şaban Oğlu Şaban,Šíleně smutná princezna
userId,,,,,,,,,,,,,,,,,,,,,
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_movie_rating_pivot.shape

(61472, 2924)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

def train_test_split(ratings, min_user_ratings=35, delete_rating_count=15):
    train = ratings.copy()
    validation = np.zeros_like(ratings)

    for user in range(ratings.shape[0]):
        user_ratings = ratings[user, :]
        n_user_ratings = np.sum(user_ratings != 0)

        if n_user_ratings >= min_user_ratings:
            indices = np.where(user_ratings != 0)[0]
            val_ratings = np.random.choice(indices, size=delete_rating_count, replace=False)

            train[user, val_ratings] = 0
            validation[user, val_ratings] = ratings[user, val_ratings]

    return train, validation


In [ ]:
train, val = train_test_split(df_movie_rating_pivot.values)

In [ ]:
print(train.shape, val.shape)

(61472, 2924) (61472, 2924)


In [ ]:
class myRecommender:
  def __init__(self, epochs = 100, n_latent_factors = 8, lmbda = 0.1, learning_rate = 0.001, w_0 = 0.02):
    self.epochs = epochs
    self.n_latent_factors = n_latent_factors
    self.lmbda = lmbda
    self.learning_rate = learning_rate
    self.w_0 = w_0

  def fit_the_data(self, Xtrain, val):
    self.n_users, self.n_movies = Xtrain.shape[0], Xtrain.shape[1]
    #print(self.n_users, self.n_movies)
    self.U = np.random.rand(self.n_users, self.n_latent_factors)
    self.V = np.random.rand(self.n_movies, self.n_latent_factors)
    self.Vt = self.V.T
    #print(self.U.shape, self.V.shape, self.Vt.shape)
    #print(self.U, "\n", self.Vt)

    self.training_process = []
    self.val_process = []

    for epoch in range(self.epochs):
      for u in range(self.n_users):
        for m in range(self.n_movies):
          #error = Xtrain[u,m] - np.dot(self.U[u, :], self.Vt[:,m])
          #print(np.dot(self.U[u, :], self.Vt[:,m]), error)
          error = Xtrain[u,m] - np.dot(self.U[u, :], self.V[m,:])

          if Xtrain[u,m] != 0:
            weight = 1
          else:
            weight = self.w_0

          self.U[u,:] += self.learning_rate * (error * self.V[m,:] - self.lmbda * self.U[u,:]) * weight
          self.V[m,:] += self.learning_rate * (error * self.U[u,:] - self.lmbda * self.V[m,:]) * weight

      train_err = sqrt(mean_squared_error(np.dot(self.U, self.Vt), Xtrain))
      test_err = sqrt(mean_squared_error(np.dot(self.U, self.Vt), val))
      print("Epoch: ", epoch , ", train rmse: ", train_err, ", test rmse: ", test_err)
      #print(self.U, self.V, np.dot(self.U, self.Vt) )
      self.training_process.append((train_err))
      self.val_process.append((test_err))

    return self

  def predict(self, Xtrain, user):
    #predicted_rating_dict = {}
    #get the prediction matrix
    rating_prediction_matrix = np.dot(self.U, self.Vt)
    #choose row for given user
    user_row = Xtrain[user, :]
    #get the index of movies that user did not watch -> rating = 0
    index = np.where(user_row == 0)[0]
    #return predicted movie ratings for items that given user did not watch
    predicted_rating = rating_prediction_matrix[user, index].flatten()
    #print(predicted_rating)
    #for i,r in zip(index, predicted_rating):
      #predicted_rating_dict[i] = r
    return predicted_rating


In [ ]:
model = myRecommender()
model.fit_the_data(train, val)

KeyboardInterrupt: 

In [ ]:
#save the model
import pickle
pickle.dump(model, open('finalized_model.sav', 'wb'))

In [ ]:
#load the model
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))

{0: 0.3332200579278565, 1: 0.026126596521623597, 2: 0.14077408986292553, 3: 0.2877391371135352, 4: -0.09125104247581159, 5: 0.2253398011896821, 6: 0.188444429288807, 7: 0.13330421251241892, 8: 0.2319402742044109, 9: 0.2169775719222345, 10: 0.13060647367928138, 11: 0.007971759226602076, 12: 0.2987581143399534, 13: 0.31673040711187533, 14: 0.3201258060563303, 15: 0.14726034331241317, 16: 0.1365570056075616, 17: -0.009757447563854783, 18: 0.18485003840532327, 19: 0.18813373544349252, 20: 0.13276836039987827, 21: 0.07753338993448024, 22: 0.2474983051790082, 23: 0.3265464492598334, 24: 0.023211309122211206, 25: 0.28609654552119906, 26: 0.35564487244118026, 27: 0.23080813331088929, 28: 0.26682873636189136, 29: 0.30497463953014786, 30: 0.27515973196503307, 31: 0.10738661041791629, 32: 0.3803231341844638, 33: 0.17013245485562487, 34: 0.24407133009782284, 35: 0.07409952437398541, 36: 0.24601150323544796, 37: 0.1742099740649286, 38: 0.21482568778525765, 39: 0.22459267960447718, 40: 0.17394082764

In [ ]:
user_id = 3
rating_predictions = loaded_model.predict(train, user_id) #get the predicted ratings for the given user

In [ ]:
#get recommendations for the given user
recommendations = sorted(rating_predictions.items(), key=lambda x: x[1], reverse=True)[:10]
recommendations

[(750, 0.6077339846829088),
 (564, 0.5981368088763257),
 (506, 0.5836951097507088),
 (337, 0.5823420192816829),
 (392, 0.5800410870657374),
 (344, 0.576056509735938),
 (212, 0.5754472398991493),
 (493, 0.5738698883545198),
 (61, 0.5725950067685768),
 (783, 0.5687657531286356)]